In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.amp import GradScaler, autocast
from torchvision.models import resnet34, ResNet34_Weights
import time

In [3]:
train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # ResNet expects 3 channels
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(p=0.7),
    transforms.RandomRotation(25),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.GaussianBlur(1),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.2),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Same here
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
])


In [4]:
train_data = datasets.ImageFolder("data/train", transform=train_transform)
val_data = datasets.ImageFolder("data/test", transform=val_transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [5]:
class EmotionResNet34(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()
        self.model = resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmotionResNet34().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=8.970677769864372e-05, weight_decay=3.162877801888854e-05)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.5)
scheduler = CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6) 
scaler = GradScaler(enabled=torch.cuda.is_available())

best_val_loss = float('inf')
best_model_wts = None
patience = 7
epochs_no_improve = 0
best_val_acc = 0

In [18]:
import optuna
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm.notebook import tqdm

def objective(trial):
    # 🔧 Suggest hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)

    # ⚙️ Setup model, optimizer, scheduler, loss
    model = EmotionResNet34().to(device)  # Replace with your model if different
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6) 
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    scaler = GradScaler(enabled=torch.cuda.is_available())

    best_val_acc = 0
    best_val_loss = float('inf')
    best_model_wts = None
    epochs = 20  # full training, no early stopping

    for epoch in tqdm(range(epochs), desc=f"Trial {trial.number}"):
        model.train()
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            with autocast(device_type='cuda'):
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()

        # 🔎 Validation
        model.eval()
        correct, total, val_loss = 0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                with autocast(device_type='cuda'):
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100 * correct / total
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = model.state_dict()

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"optuna_best_model_trial_{trial.number}.pth")
            print(f"💾 Trial {trial.number}: Saved model with acc {val_acc:.2f}%")

    # 🔄 Restore best model weights
    if best_model_wts is not None:
        model.load_state_dict(best_model_wts)

    return best_val_acc


In [19]:
n_trials = 30
study = optuna.create_study(direction='maximize')
pbar = tqdm(range(n_trials), desc="Optuna Trials")

for _ in pbar:
    study.optimize(objective, n_trials=1, catch=(Exception,))
    pbar.set_postfix_str(f"Best Acc: {study.best_value:.2f}%")

print("🎯 Best hyperparameters:", study.best_params)
print("🎯 Best validation accuracy:", study.best_value)

[I 2025-07-02 01:33:14,774] A new study created in memory with name: no-name-a114d4e8-44b5-4a81-ba38-3e4ea60d0d51


Optuna Trials:   0%|          | 0/30 [00:00<?, ?it/s]

Trial 0:   0%|          | 0/20 [00:00<?, ?it/s]

💾 Trial 0: Saved model with acc 45.37%
💾 Trial 0: Saved model with acc 49.61%
💾 Trial 0: Saved model with acc 52.15%
💾 Trial 0: Saved model with acc 53.21%
💾 Trial 0: Saved model with acc 54.80%
💾 Trial 0: Saved model with acc 56.30%
💾 Trial 0: Saved model with acc 57.74%
💾 Trial 0: Saved model with acc 59.21%
💾 Trial 0: Saved model with acc 59.40%
💾 Trial 0: Saved model with acc 60.55%
💾 Trial 0: Saved model with acc 60.79%
💾 Trial 0: Saved model with acc 61.65%
💾 Trial 0: Saved model with acc 61.89%


[I 2025-07-02 01:50:16,973] Trial 0 finished with value: 62.32142857142857 and parameters: {'lr': 3.004139144576653e-05, 'weight_decay': 2.2208210359455036e-05}. Best is trial 0 with value: 62.32142857142857.


...

[I 2025-07-02 09:54:34,868] Trial 28 finished with value: 63.79761904761905 and parameters: {'lr': 4.670431807795256e-05, 'weight_decay': 7.293055518181136e-06}. Best is trial 7 with value: 64.57142857142857.


Trial 29:   0%|          | 0/20 [00:00<?, ?it/s]

💾 Trial 29: Saved model with acc 49.54%
💾 Trial 29: Saved model with acc 50.93%
💾 Trial 29: Saved model with acc 58.39%
💾 Trial 29: Saved model with acc 60.29%
💾 Trial 29: Saved model with acc 60.44%
💾 Trial 29: Saved model with acc 61.71%
💾 Trial 29: Saved model with acc 62.32%
💾 Trial 29: Saved model with acc 62.98%
💾 Trial 29: Saved model with acc 63.01%


[I 2025-07-02 10:11:18,061] Trial 29 finished with value: 63.01190476190476 and parameters: {'lr': 0.0002240851862405931, 'weight_decay': 2.5982886829592898e-05}. Best is trial 7 with value: 64.57142857142857.


🎯 Best hyperparameters: {'lr': 8.970677769864372e-05, 'weight_decay': 3.162877801888854e-05}
🎯 Best validation accuracy: 64.57142857142857


In [8]:
for epoch in range(50):
    epoch_start = time.time()
    model.train()
    total_loss = 0

    print(f"\n🔁 Epoch {epoch+1}/50")

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        with autocast(device_type='cuda'):
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

        if (batch_idx + 1) % 30 == 0 or (batch_idx + 1) == len(train_loader):
            avg_loss = total_loss / (batch_idx + 1)
            percent = 100 * (batch_idx + 1) / len(train_loader)
            print(f"  🟩 Batch {batch_idx+1:03d}/{len(train_loader)} ({percent:.1f}%) - Avg Loss: {avg_loss:.4f}", end="\r")

    avg_train_loss = total_loss / len(train_loader)

    # Validation phase
    model.eval()
    correct = 0
    total = 0
    val_loss = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            with autocast(device_type='cuda'):
                outputs = model(images)
                loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_loss /= len(val_loader)
    acc = 100 * correct / total

    epoch_time = time.time() - epoch_start
    mins, secs = divmod(epoch_time, 60)

    print(f"Epoch {epoch+1}/50 | Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {acc:.2f}% | ⏱ Time: {int(mins)}m {int(secs)}s")

    scheduler.step()

    # Save best loss model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    # Save best accuracy model
    if acc > best_val_acc:
        best_val_acc = acc
        torch.save(model.state_dict(), "best_emotion_model_resnet34.pth")
        print("💾 Saved new best model based on accuracy.")

    # Early stopping
    # if epochs_no_improve == patience:
    #     print(f"\n⛔ Early stopping triggered at epoch {epoch+1}")
    #     break

# Final save
model.load_state_dict(best_model_wts)
torch.save(model.state_dict(), "emotion_model_resnet34.pth")
print("\n✅ Best model loaded and saved to emotion_model_resnet34.pth")


🔁 Epoch 1/50
Epoch 1/50 | Train Loss: 1.0083 | Val Loss: 1.2771 | Val Acc: 62.65% | ⏱ Time: 0m 53s

🔁 Epoch 2/50
Epoch 2/50 | Train Loss: 0.9933 | Val Loss: 1.3020 | Val Acc: 62.67% | ⏱ Time: 0m 55s

🔁 Epoch 3/50
Epoch 3/50 | Train Loss: 0.9713 | Val Loss: 1.2569 | Val Acc: 64.51% | ⏱ Time: 0m 55s
💾 Saved new best model based on accuracy.

🔁 Epoch 4/50
Epoch 4/50 | Train Loss: 0.9625 | Val Loss: 1.3038 | Val Acc: 62.11% | ⏱ Time: 0m 55s

🔁 Epoch 5/50
Epoch 5/50 | Train Loss: 0.9383 | Val Loss: 1.2570 | Val Acc: 64.49% | ⏱ Time: 0m 56s

🔁 Epoch 6/50
Epoch 6/50 | Train Loss: 0.9161 | Val Loss: 1.2943 | Val Acc: 63.37% | ⏱ Time: 0m 56s

🔁 Epoch 7/50
Epoch 7/50 | Train Loss: 0.9085 | Val Loss: 1.2817 | Val Acc: 63.93% | ⏱ Time: 0m 56s

🔁 Epoch 8/50
Epoch 8/50 | Train Loss: 0.8889 | Val Loss: 1.3062 | Val Acc: 63.31% | ⏱ Time: 0m 57s

🔁 Epoch 9/50
Epoch 9/50 | Train Loss: 0.8764 | Val Loss: 1.3002 | Val Acc: 63.67% | ⏱ Time: 0m 58s

🔁 Epoch 10/50
Epoch 10/50 | Train Loss: 0.8526 | Val Loss

KeyboardInterrupt: 